# 04. Performance Profiling

We can profile the performance with a 3D FDTD simulation:

## Imports

In [ ]:
import matplotlib.pyplot as plt
from line_profiler import LineProfiler

import fdtd
import fdtd.backend as bd

## Set Backend

Let's profile the impact of the backend. These are the possible backends:

- ``numpy`` (defaults to float64 arrays)
- ``torch`` (defaults to float64 tensors)
- ``torch.float32``
- ``torch.float64``
- ``torch.cuda`` (defaults to float64 tensors)
- ``torch.cuda.float32``
- ``torch.cuda.float64``

In [ ]:
fdtd.set_backend("numpy")

In general, the ``numpy`` backend is preferred for standard CPU calculations
with `"float64"` precision as it is slightly faster than ``torch`` backend on CPU. However, a significant performance improvement can be obtained by choosing ``torch.cuda`` on large enough grids.

Note that, in FDTD, ``float64`` precision is generally preferred over ``float32`` to ensure numerical stability and prevent numerical dispersion. If this is of no concern to you, you can opt for ``float32`` precision, which especially on a GPU might yield a significant performance boost.

## Constants

In [ ]:
WAVELENGTH = 1550e-9
SPEED_LIGHT: float = 299_792_458.0  # [m/s] speed of light

## Setup Simulation

create FDTD Grid

In [ ]:
N = 100

grid = fdtd.Grid(
    (N, N, N),
    grid_spacing=0.05 * WAVELENGTH,
    permittivity=1.0,
    permeability=1.0,
)

add boundaries

In [ ]:
# x boundaries
grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")

# y boundaries
grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")

# z boundaries
grid[:, :, 0:10] = fdtd.PML(name="pml_zlow")
grid[:, :, -10:] = fdtd.PML(name="pml_zhigh")

add sources

In [ ]:
grid[10+N//10:10+N//10, :, :] = fdtd.PlaneSource(
    period=WAVELENGTH / SPEED_LIGHT, name="source"
)

add objects

In [ ]:
grid[10+N//5:4*N//5-10, 10+N//5:4*N//5-10, 10+N//5:4*N//5-10] = fdtd.Object(permittivity=2.5, name="center_object")

grid summary

In [ ]:
print(grid)

## Setup LineProfiler

create and enable profiler

In [ ]:
profiler = LineProfiler()
profiler.add_function(grid.update_E)
profiler.enable()

## Run Simulation

run simulation

In [ ]:
grid.run(50, progress_bar=True)

## Profiler Results

print profiler summary

In [ ]:
profiler.print_stats()

## Visualization

In [ ]:
plt.figure()
grid.visualize(z=N//2)